In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

# from osgeo import gdal
from os.path import join

INPUT_FOLDER = "D:/Donnees/Others/geographic/bangladesh"

## Simulation de l'historique des inondation

Dans cette cellule, nous allons coder une petite simulation qui va générer des données factices d'inondation. Cette simulation va calculer une série temporelle des inondations frappant les écoles.

In [2]:
school_count = 1000
x_center = 100
y_center = 25
x_width = 10
y_width = 5

filepath = join(INPUT_FOLDER, "dummy_schools_1.csv")

schools = pd.DataFrame(
    index=pd.Index(name="school", data=[f"school {i}" for i in range(school_count)]),
    data={
        "x": np.random.rand(school_count),
        "y": np.random.rand(school_count)
    }
)
years = np.arange(2020, 2101, dtype=int)

serie = pd.DataFrame()
tm = 0.0
tp = 0.0
for year in years:
    tp = (year - years.min()) / (years.max() - years.min())
    
    impacted = schools.loc[(schools["y"] >= tm) & (schools["y"] < tp), :].copy()
    impacted["year"] = year
    
    serie = pd.concat((serie, impacted.reset_index()), axis=0)
    tm = tp

# post-processing
serie["date"] = serie["year"].apply(lambda x: dt.date(year=x, month=1, day=1))
serie["x"] = x_center + x_width * (serie["x"] - 0.5)
serie["y"] = y_center + y_width * (serie["y"] - 0.5)
serie = serie.sort_values(by=["year", "school"], ascending=True)[["date", "school", "x", "y"]]

serie.to_csv(filepath, sep=",", header=True, index=False)
serie.info()
serie.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 8
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1000 non-null   object 
 1   school  1000 non-null   object 
 2   x       1000 non-null   float64
 3   y       1000 non-null   float64
dtypes: float64(2), object(2)
memory usage: 39.1+ KB


,date,school,x,y
1,2021-01-01,school 120,95.300972,22.525006
2,2021-01-01,school 266,104.796929,22.541841
3,2021-01-01,school 268,97.139299,22.513946
4,2021-01-01,school 277,100.587131,22.525123
0,2021-01-01,school 37,97.940749,22.539929


The second simulation is based from the schools coordinates. It takes a sample of the set of schools and assigns them randomly to a flood date. This allows for more consistent beta-testing in the QGIS temporal animation.

In [3]:
schools = pd.read_csv(join(INPUT_FOLDER, "schools.csv"), header=0, sep=";", decimal=".")

schools = (
    schools
    .where(schools["x_clean"] != "#VALEUR!")
    .where(schools["y_clean"] != "#VALEUR!")
    .dropna()
    .applymap(lambda x: x.replace("(", "").replace(")", ""))
    .astype(float)
    .sample(frac=0.1)
)

schools["date"] = (
    pd
    .date_range(
        name="date",
        start=dt.date(2020, 1, 1),
        end=dt.date(2100, 1, 1),
        periods=len(schools)
    )
    .to_frame()
    ["date"]
    .apply(lambda x: dt.date(x.year, x.month, x.day))
    .values
)

schools["school"] = [f"school {i}" for i in range(len(schools))]

schools = (
    schools
    .sort_values(by="date")
    .rename(columns={"x_clean": "x", "y_clean": "y"})
    [["date", "school", "x", "y"]]
)

schools.to_csv(join(INPUT_FOLDER, "dummy_school_flood.csv"), sep=",", header=True, index=False)
schools.info()
schools

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7797 entries, 24734 to 62970
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    7797 non-null   object 
 1   school  7797 non-null   object 
 2   x       7797 non-null   float64
 3   y       7797 non-null   float64
dtypes: float64(2), object(2)
memory usage: 304.6+ KB


,date,school,x,y
24734,2020-01-01,school 0,90.790164,24.282569
49058,2020-01-04,school 1,88.976037,24.101200
48341,2020-01-08,school 2,89.266506,22.667670
3406,2020-01-12,school 3,90.524481,22.396471
65355,2020-01-15,school 4,89.425705,25.083335
...,...,...,...,...
9772,2099-12-17,school 7792,90.856900,23.458379
18221,2099-12-20,school 7793,89.577725,23.581507
23208,2099-12-24,school 7794,89.816124,24.982133
34075,2099-12-28,school 7795,90.515618,23.727532
